In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GRU, Activation, Dense, Dropout, Input, Embedding



In [2]:
import nltk # natural language tokeniser
nltk.download('punkt') # sentence tokeniser

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from sklearn.preprocessing import MultiLabelBinarizer #multi label questions

In [4]:
from bs4 import BeautifulSoup  # Python library for pulling data out of HTML and XML files.
from urllib.request import urlopen

In [5]:
!wget 'https://www.dropbox.com/s/5721wcs2guuykzl/stacksample.zip?dl=0'# downloading data from dropbox

--2023-07-25 07:11:36--  https://www.dropbox.com/s/5721wcs2guuykzl/stacksample.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/5721wcs2guuykzl/stacksample.zip [following]
--2023-07-25 07:11:36--  https://www.dropbox.com/s/raw/5721wcs2guuykzl/stacksample.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca34da47d12b29ac23286981626.dl.dropboxusercontent.com/cd/0/inline/CAgAVpJj3zemWp3m8dXZ5rR-Nr7A6jYBTNQJGbRY1vzevP3cjpOgLSPXimmMlcM1eFl4CIiJXO2eAbr05KkmvzOpAVK-mkOyogz2ssOLcVQ6Fd1572d7YTsTZBNh1m86jIwO4M2IKvNc8puq7-z4eKYM8dTc6OC60a0UD_DDpl63lA/file# [following]
--2023-07-25 07:11:36--  https://uca34da47d12b29ac23286981626.dl.dropboxusercontent.com/cd/0/inline/CAgAVpJj3zemWp3m8dXZ5rR-Nr7A6jYBTNQJGbRY1vzevP3cjpOgLSPXimmMlcM

In [6]:
!unzip stacksample.zip?dl=0 # unzipping the sample file into question, answer and tag files

Archive:  stacksample.zip?dl=0
  inflating: Answers.csv             
  inflating: Questions.csv           
  inflating: Tags.csv                


In [7]:
!rm stacksample.zip?dl=0 # now remove the zip file as it is no longer required

In [8]:
consideredMaxId = 1250000

In [9]:
questionList = pd.read_csv('/content/Questions.csv', usecols=["Id", "Body"], encoding = "ISO-8859-1")

In [10]:
questionList

,Id,Body
0,80,<p>I've written a database generation script i...
1,90,<p>Are there any really good tutorials explain...
2,120,<p>Has anyone got experience creating <strong>...
3,180,<p>This is something I've pseudo-solved many t...
4,260,<p>I have a little game written in C#. It uses...
...,...,...
1264211,40143210,<p>I am building a custom MVC project and I ha...
1264212,40143300,<p>The API docs show that you should be able t...
1264213,40143340,<p>Under minifyEnabled I changed from false to...
1264214,40143360,<p>I have input which I use to filter my array...


In [11]:
tagList = pd.read_csv('/content/Tags.csv',usecols = ["Id", "Tag"], encoding = "ISO-8859-1")


In [12]:
tagList

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn
...,...,...
3750989,40143360,javascript
3750990,40143360,vue.js
3750991,40143380,npm
3750992,40143380,mocha


In [13]:
print('PRIOR')

print(questionList.shape, tagList.shape)
questionList = questionList.loc[questionList['Id'] < consideredMaxId] #answersList = answersList.loc[answersList['Id'] < consideredMaxId]

tagList = tagList.loc[tagList['Id'] < consideredMaxId]
print('POST')

print(questionList.shape, tagList.shape)

PRIOR
(1264216, 2) (3750994, 2)
POST
(23022, 2) (66396, 2)


In [14]:
def GetProperData(txt):
	soup = BeautifulSoup(txt,'lxml')
	paras=soup.find_all('p')
	paras=[p.text for p in paras]
	return paras

In [15]:
questionList.sort_values(by=['Id']) #sorting questionlist by Id
questionList['Body'] = questionList.apply(lambda x: GetProperData(x['Body']), axis=1)
questionList['Body'] = questionList.Body.apply(lambda x: ' '.join(x))

<ipython-input-15-0ff476610fe8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questionList['Body'] = questionList.apply(lambda x: GetProperData(x['Body']), axis=1)
<ipython-input-15-0ff476610fe8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questionList['Body'] = questionList.Body.apply(lambda x: ' '.join(x))


In [16]:
questionList.head()

,Id,Body
0,80,I've written a database generation script in S...
1,90,Are there any really good tutorials explaining...
2,120,Has anyone got experience creating SQL-based A...
3,180,This is something I've pseudo-solved many time...
4,260,I have a little game written in C#. It uses a ...


In [17]:
questionList._get_value(0, 'Body', takeable=False)

"I've written a database generation script in SQL and want to execute it in my Adobe AIR application: I execute this in Adobe AIR using the following methods: No errors are generated, however only tRole exists. It seems that it only looks at the first query (up to the semicolon- if I remove it, the query fails). Is there a way to call multiple queries in one statement?"

In [18]:
tagList.isnull().sum() # find if null exists

Id      0
Tag    21
dtype: int64

In [19]:
tagList["Tag"].fillna("_null", inplace = True) # replece null with _null

<ipython-input-19-cc36c0d79f5f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tagList["Tag"].fillna("_null", inplace = True) # replece null with _null


In [20]:
tagList.isnull().sum() #check if all nulls replaced

Id     0
Tag    0
dtype: int64

In [21]:

# find only get top 10 most frequent names
n = 10
print(tagList['Tag'].value_counts())
top_n_Tag = tagList['Tag'].value_counts()[:n].index.tolist()
print(top_n_Tag)

c#                  3148
.net                1930
java                1649
asp.net             1497
javascript          1164
                    ... 
gcc-warning            1
gsm                    1
hashchange             1
precision-recall       1
exc-bad-access         1
Name: Tag, Length: 6537, dtype: int64
['c#', '.net', 'java', 'asp.net', 'javascript', 'c++', 'php', 'python', 'sql', 'sql-server']


In [22]:
tagList_top_n = tagList[tagList['Tag'].isin(top_n_Tag)]

In [23]:
tagList = tagList_top_n
print(tagList)

            Id      Tag
7          120      sql
8          120  asp.net
14         260       c#
15         260     .net
18         330      c++
...        ...      ...
66377  1249550     .net
66380  1249670   python
66382  1249680       c#
66383  1249680     .net
66386  1249750      c++

[14026 rows x 2 columns]


In [24]:

df_q = questionList
print(df_q)
a = tagList['Id']
df_q = df_q[df_q['Id'].isin(a)]
print(df_q)
questionList = df_q

            Id                                               Body
0           80  I've written a database generation script in S...
1           90  Are there any really good tutorials explaining...
2          120  Has anyone got experience creating SQL-based A...
3          180  This is something I've pseudo-solved many time...
4          260  I have a little game written in C#. It uses a ...
...        ...                                                ...
23017  1249670  I'm becoming acquainted with python and am cre...
23018  1249680  Using C#.Net On my form i have a button that o...
23019  1249750  Given two stream-oriented I/O objects in Asio,...
23020  1249840  I have a UINavigationController onto which I p...
23021  1249890  I'm an experienced software developer (mostly ...

[23022 rows x 2 columns]
            Id                                               Body
2          120  Has anyone got experience creating SQL-based A...
4          260  I have a little game written in C#

In [25]:
def add_tags(question_id):
    return tagList[tagList['Id'] == question_id['Id']].Tag.values

temp = tagList.apply(add_tags, axis=1) # can check the datatype of temp variable
print(type(temp))

<class 'pandas.core.series.Series'>


In [26]:
tagList=pd.concat([tagList, temp.rename('Tags')], axis=1)
tagList

,Id,Tag,Tags
7,120,sql,"[sql, asp.net]"
8,120,asp.net,"[sql, asp.net]"
14,260,c#,"[c#, .net]"
15,260,.net,"[c#, .net]"
18,330,c++,[c++]
...,...,...,...
66377,1249550,.net,[.net]
66380,1249670,python,[python]
66382,1249680,c#,"[c#, .net]"
66383,1249680,.net,"[c#, .net]"


In [27]:
tagList = tagList.drop(['Tag'], axis = 1)

In [28]:
tagList = tagList.drop_duplicates(subset="Id", keep="first", inplace=False)

In [29]:
questionList = pd.merge(questionList, tagList, on='Id')

In [30]:
questionList.isnull().sum() #checking for null

Id      0
Body    0
Tags    0
dtype: int64

In [31]:
sent_lens=[]
for sent in questionList['Body']:
    sent_lens.append(len(word_tokenize(sent)))

In [32]:
max(sent_lens)

1223

In [33]:
for percentile in [0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97]:
  print('For percentile:', percentile, ":", np.quantile(sent_lens, percentile))


For percentile: 0.81 : 160.0
For percentile: 0.82 : 164.0
For percentile: 0.83 : 168.0
For percentile: 0.84 : 173.0
For percentile: 0.85 : 178.0
For percentile: 0.86 : 182.0
For percentile: 0.87 : 187.0
For percentile: 0.88 : 193.0
For percentile: 0.89 : 200.0
For percentile: 0.91 : 216.14999999999964
For percentile: 0.92 : 227.0
For percentile: 0.93 : 237.0
For percentile: 0.94 : 249.0
For percentile: 0.95 : 264.0
For percentile: 0.96 : 283.0
For percentile: 0.97 : 308.0499999999993


In [34]:
max_len = int(np.quantile(sent_lens, 0.85))
print(max_len)

178


In [35]:
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(questionList['Body'])

In [36]:
multilabel = MultiLabelBinarizer()
multilabel.fit_transform(questionList['Tags'])

array([[0, 1, 0, ..., 0, 1, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [37]:
multilabel.classes_

array(['.net', 'asp.net', 'c#', 'c++', 'java', 'javascript', 'php',
       'python', 'sql', 'sql-server'], dtype=object)

In [38]:
Q_train, Q_test = train_test_split(questionList,test_size=0.2,random_state=2)
x_train = Q_train['Body']
y_train = Q_train['Tags']
x_test  = Q_test['Body']
y_test  = Q_test['Tags']

In [39]:
#input to model Train_X = sequences_matrix_train
sequences_train = tok.texts_to_sequences(x_train)
sequences_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_len)


In [40]:
#input to model Test_X = sequences_matrix_test
sequences_test = tok.texts_to_sequences(x_test)
sequences_matrix_test = sequence.pad_sequences(sequences_test, maxlen=max_len)
sequences_matrix_test.shape

(2374, 178)

In [41]:
vocab_len = len(tok.index_word.keys())
vocab_len

30320

In [42]:
#Output to model Train_Y = y_train_multiLevel
y_train_multiLevel = multilabel.transform(y_train)
y_train_multiLevel.shape

(9492, 10)

In [43]:
outputNode = len(multilabel.classes_)
max_len, outputNode

(178, 10)

In [44]:
multilabel.classes_

array(['.net', 'asp.net', 'c#', 'c++', 'java', 'javascript', 'php',
       'python', 'sql', 'sql-server'], dtype=object)

In [45]:
#Output to model Test_Y = y_test_multiLevel
y_test_multiLevel = multilabel.transform(y_test)

In [46]:
def RNN(): # final model
    inputs = Input(name='inputs',shape=[max_len])

    layer = Embedding(vocab_len+1,1500,input_length=max_len,
                      mask_zero=True)(inputs)
    layer = GRU(400)(layer)
    layer = Dense(200,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dense(100,name='FC2')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(outputNode,name='out_layer')(layer)
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [47]:
model = RNN()
model.summary()

model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 178)]             0         
                                                                 
 embedding (Embedding)       (None, 178, 1500)         45481500  
                                                                 
 gru (GRU)                   (None, 400)               2282400   
                                                                 
 FC1 (Dense)                 (None, 200)               80200     
                                                                 
 activation (Activation)     (None, 200)               0         
                                                                 
 FC2 (Dense)                 (None, 100)               20100     
                                                                 
 activation_1 (Activation)   (None, 100)               0     

In [48]:
model.fit(sequences_matrix_train,y_train_multiLevel,batch_size=50,
          epochs=3,validation_data=(sequences_matrix_test,y_test_multiLevel))


Epoch 1/3
190/190 [==============================] - 1105s 6s/step - loss: 0.3798 - accuracy: 0.1671 - val_loss: 0.3363 - val_accuracy: 0.2110
Epoch 2/3
190/190 [==============================] - 1088s 6s/step - loss: 0.2461 - accuracy: 0.4778 - val_loss: 0.2001 - val_accuracy: 0.5775
Epoch 3/3
190/190 [==============================] - 1103s 6s/step - loss: 0.1555 - accuracy: 0.6956 - val_loss: 0.2013 - val_accuracy: 0.5952


In [49]:


predictions=model.predict(sequences_matrix_test)

from sklearn.metrics import roc_auc_score

roc_auc_score(y_test_multiLevel,predictions)

75/75 [==============================] - 72s 931ms/step


0.9146971398671612